In [1]:
import functions as fxn
import numpy as np
import pandas as pd

In [ ]:
columns = ['INSTNM', 'HIGHDEG', 'CONTROL', 'REGION', 'LOCALE', 'LO_INC_DEBT_N', 'MD_INC_DEBT_N', 'HI_INC_DEBT_N',
           'LOAN_EVER', 'PELL_EVER', 'PCTPELL', 'ICLEVEL', 'CURROPER', 'TUITFTE', 'CDR3', 'INEXPFTE']
sheets = fxn.read_in_CollegeScorecard(columns)
full_df = fxn.concatenate_all_sheets(sheets)
full_df_nonan = fxn.quick_clean(full_df, columns=['PCTPELL', 'CDR3'])

In [ ]:
fxn.back_fill_from_year(full_df_nonan, 2017).head()